# Helpdesk Medium

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.0     v purrr   0.3.4
v tibble  3.0.1     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Password? ·········


## 6.
List the Company name and the number of calls for those companies with more than 18 calls.

```
+------------------+----+
| Company_name     | cc |
+------------------+----+
| Gimmick Inc.     | 22 |
| Hamming Services | 19 |
| High and Co.     | 20 |
+------------------+----+
```

In [2]:
shift <- dbReadTable(con, 'Shift')
staff <- dbReadTable(con, 'Staff')
issue <- dbReadTable(con, 'Issue')
shift_type <- dbReadTable(con, 'Shift_type')
level <- dbReadTable(con, 'Level')
customer <- dbReadTable(con, 'Customer')
caller <- dbReadTable(con, 'Caller')

In [3]:
issue %>%
    inner_join(caller, by=c(Caller_id="Caller_id")) %>%
    inner_join(customer, by=c(Company_ref="Company_ref")) %>%
    group_by(Company_name) %>%
    summarise(cc=n()) %>%
    filter(cc>18) %>%
    arrange(Company_name)

Company_name,cc
<chr>,<int>
Gimmick Inc.,22
Hamming Services,19
High and Co.,20


## 7.
Find the callers who have never made a call. Show first name and last name

```
+------------+-----------+
| first_name | last_name |
+------------+-----------+
| David      | Jackson   |
| Ethan      | Phillips  |
+------------+-----------+
```

In [4]:
caller %>%
    left_join(issue, by=c(Caller_id="Caller_id")) %>%
    group_by(Caller_id, First_name, Last_name) %>%
    tally(wt=!is.na(Call_ref)) %>%
    ungroup %>%
    filter(n<1) %>%
    select(First_name, Last_name) %>%
    arrange(First_name)

First_name,Last_name
<chr>,<chr>
David,Jackson
Ethan,Phillips


## 8.
For each customer show: Company name, contact name, number of calls where the number of calls is fewer than 5

```
+--------------------+------------+-----------+----+
| Company_name       | first_name | last_name | nc |
+--------------------+------------+-----------+----+
| Pitiable Shipping  | Ethan      | McConnell |  4 |
| Rajab Group        | Emily      | Cooper    |  4 |
| Somebody Logistics | Ethan      | Phillips  |  2 |
+--------------------+------------+-----------+----+
```

In [5]:
issue %>%
    inner_join(caller, by=c(Caller_id="Caller_id")) %>%
    inner_join(customer, by=c(Company_ref="Company_ref")) %>%
    inner_join(caller, by=c(Contact_id="Caller_id")) %>%
    group_by(Company_name, Company_ref.x, First_name.y, Last_name.y) %>%
    tally(wt=!is.na(Call_ref), name="nc") %>%
    ungroup %>%
    filter(nc<5) %>%
    arrange(Company_name) %>%
    select(Company_name, First_name.y, Last_name.y, nc)

Company_name,First_name.y,Last_name.y,nc
<chr>,<chr>,<chr>,<int>
Pitiable Shipping,Ethan,McConnell,4
Rajab Group,Emily,Cooper,4
Somebody Logistics,Ethan,Phillips,2


## 9.
For each shift show the number of staff assigned. Beware that some roles may be NULL and that the same person might have been assigned to multiple roles (The roles are 'Manager', 'Operator', 'Engineer1', 'Engineer2').

```
+------------+------------+----+
| Shift_date | Shift_type | cw |
+------------+------------+----+
| 2017-08-12 | Early      |  4 |
| 2017-08-12 | Late       |  4 |
| 2017-08-13 | Early      |  3 |
| 2017-08-13 | Late       |  2 |
| 2017-08-14 | Early      |  4 |
| 2017-08-14 | Late       |  4 |
| 2017-08-15 | Early      |  4 |
| 2017-08-15 | Late       |  4 |
| 2017-08-16 | Early      |  4 |
| 2017-08-16 | Late       |  4 |
+------------+------------+----+
```

In [6]:
SELECT agg."Shift_date", agg."Shift_type", COUNT(agg.p) cw FROM
(SELECT DISTINCT a."Shift_date", a."Shift_type", a.p FROM
  (SELECT "Shift_date", "Shift_type", "Manager" p FROM "Shift" 
    UNION SELECT "Shift_date", "Shift_type", "Operator" p FROM "Shift" 
    UNION SELECT "Shift_date", "Shift_type", "Engineer1" p FROM "Shift" 
    UNION SELECT "Shift_date", "Shift_type", "Engineer2" p FROM "Shift" 
  ) a
) AS agg
GROUP BY agg."Shift_date", agg."Shift_type"
ORDER BY agg."Shift_date", agg."Shift_type";



ERROR: Error in parse(text = x, srcfile = src): <text>:1:8: unexpected symbol
1: SELECT agg.
           ^


In [ ]:
shift %>% 
    pivot_longer(Manager:Engineer2, names_to="role", values_to="who") %>%
    distinct(Shift_date, Shift_type, who) %>%
    group_by(Shift_date, Shift_type) %>%
    tally(!is.na(who), name='cw')

## 10.
Caller 'Harry' claims that the operator who took his most recent call was abusive and insulting. Find out who took the call (full name) and when.

```
+------------+-----------+---------------------+
| first_name | last_name | call_date           |
+------------+-----------+---------------------+
| Emily      | Best      | 2017-08-16 10:25:00 |
+------------+-----------+---------------------+
```

In [ ]:
caller %>%
    inner_join(issue, by=c(Caller_id="Caller_id")) %>%
    inner_join(staff, by=c(Taken_by="Staff_code")) %>%
    filter(First_name.x=='Harry') %>%
    arrange(desc(Call_date)) %>%
    select(First_name.y, Last_name.y, Call_date) %>%
    slice(1)

In [ ]:
dbDisconnect(con)